In [2]:
from newspaper import Article
import urllib.request
from bs4 import BeautifulSoup
import os
import html
import glob
import codecs

In [3]:
#-*- coding: utf-8 -*-
output_path = "urls.csv"

f = codecs.open(output_path,"r","UTF-8")
url_lists_with_newline = f.readlines()

urls_list = []
urls_to_article = []
dates = []
reporters = []
titles = []
articles = []
sitenames = []
times=[]
id = []
for url_element_with_newline in url_lists_with_newline:
    urls_list.append(url_element_with_newline.strip())
    

In [3]:
numbering=1
for url in urls_list:
    url_without_http = "".join(url.split("http://")[1:])#.split("/")[0]   
    if "www.newdaily.co.kr" in url_without_http:
        url = "http://" + url_without_http
        article_url = urllib.request.urlopen(url).read()
        soup=BeautifulSoup(article_url, "lxml")
        content = Article(url, language='ko')
        content.download()
        content.parse()
        
        if content.publish_date is None:
            date = soup.find("meta", {"property":"article:published_time"})
            date1 = soup.find("div", {"class":"news-info"})
            date2 = soup.find("p", {"class":"arvdate"})
            date3 = soup.find("div",{"class":"writeInfo"})
            
            #print (date2)
            
            if date is not None:
                dates.append(date["content"].split("T")[0])
                times.append("".join(date["content"].split("T")[1]).split("+")[0])
            elif date1 is not None:
                dates.append("".join(date1.text.split("입력시간")[1]).split(" ")[0].strip().replace("|",""))
                times.append("".join(date1.text.split("입력시간")[1]).split(" ")[1].strip())
            elif date2 is not None:
                dates.append("".join(date2.text.split("최종편집")[1]).strip().split(" ")[0])
                times.append("".join(date2.text.split("최종편집")[1]).strip().split(" ")[1])
            elif date3 is not None:
                dates.append("".join(date3.text.split("최종편집")[1]).strip().split(" ")[0].replace(",","-"))
                times.append("".join(date3.text.split("최종편집")[1]).strip().split(" ")[1])
            
            #else:
            #    dates.append(url) 
        else:
            t = content.publish_date
            date = t.strftime("%Y-%m-%d")
            dates.append(date)
            
            time = t.strftime("%H:%M:%S")
            times.append(time)
            
        if len(content.title) == 0:
            title = soup.find("meta", {"property":"og:title"})
            titles.append(title["content"] if title else url)
        else:
            titles.append(content.title)
            
        if len(content.authors) == 0:
            reporter = soup.find("meta", {"property":"dable:author"})
            reporter1 = soup.find("div", {"class":"news-info"})
            reporter2 = soup.find("p", {"class":"arvdate"})
            reporter3 = soup.find("div",{"class":"writeInfo"})
            
            if reporter is not None:
                reporters.append(reporter["content"][0:3].strip())
            elif reporter1 is not None:
                reporters.append("".join(reporter1.text.split("기자")[0])[0:4].strip())
            elif reporter2 is not None:
                reporters.append("".join(reporter2.text.split("기자")[0])[0:4].strip())
            elif reporter3 is not None:
                reporters.append("".join(reporter3.text.split("기자")[0])[0:4].strip())
        else:
            reporters.append(str(content.authors).replace("기자","").strip())
            
        if len(content.text) == 0:
            article = soup.find("div", {"itemprop":"articleBody"})
            articles.append(article.text.strip().replace("\xa0", " ").replace('\n', "").replace("\r", "").replace("\t", "") if article else "No meta article given")
        else:
            articles.append(content.text.strip().replace("\xa0", " ").replace('\n', "").replace("\r", "").replace("\t", "").replace("\u200b",""))
            
        sitenames.append("뉴데일리경제")
        id.append(numbering)
        numbering += 1


In [4]:
dates_ = []
for item in dates:
    dates_.append(item.replace(".","-"))

In [5]:
print (times)

['09:49:40', '18:30:28', '18:23:45', '12:45:43', '12:38:33', '11:06:13', '11:06:13', '11:06:13', '18:11:04', '17:47:47', '17:47:47', '12:07:20', '10:59:22', '12:08:45', '06:50:11', '07:00:11', '18:47:52', '16:28:02', '11:50:43', '12:40:39', '20:50:19', '22:55:22', '07:17:17', '21:00:42', '21:00:42', '07:08:44', '07:08:44', '11:02:43', '11:02:43', '14:33:15', '14:33:15', '12:54:55', '13:57:10', '13:57:10', '16:12:59', '16:09:20', '16:47:41', '16:47:41', '08:08:51', '18:53:07', '15:56:55', '22:22:13', '17:36:19', '17:36:19', '10:35:18', '11:30:34', '16:56:52', '16:56:52', '02:35:15', '13:32:56', '10:28:43', '10:28:43', '10:28:43', '16:10:19', '16:28:20', '17:27:05', '18:17:56', '11:44:46', '12:02:48', '20:53:49', '19:06:49', '09:18:47', '18:36:06', '20:51:04', '19:31:54', '17:50:09', '11:14:30', '12:24:38', '22:18:49', '19:31:54', '19:31:54', '19:31:54', '17:50:09', '17:50:09', '17:50:09', '11:14:30', '12:24:38', '12:24:38', '12:24:38', '12:24:38', '22:18:49', '15:38:23', '15:38:23', '15

In [6]:
print (articles[0])

신당 창당, 나중 문제로 못박아…원내에 집중할 듯 김용태 "비대위는 과정일 뿐 탄핵이 중요…입장 밝혀야" "대통령 출당 등 이뤄졌으면 탈당할 이유 없었다…새누리당 입장 모호"무소속 김용태 의원이 "지금 가장 중요한 것은 탄핵 절차에 착수하는 것"이라고 말했다.새누리당의 차기 지도부 구성과 신당 창당 문제는 부수적인 문제일 뿐이라는 설명으로, 우선 새누리당 의원들이 탄핵에 대한 입장을 분명히 밝혀야 한다는 주장이다.김용태 의원은 23일 YTN 라디오 〈신율의 출발 새아침〉에 출연해 "대통령의 직무를 정지시켜 예측 가능성을 높여야 한다"면서 이같이 밝혔다.김 의원은 "현재 공직사회나 경제계는 미래를 예측할 수 없으므로 아무 계획을 세울 수 없다고 한다"면서 "지금은 빨리 대통령의 직무를 정지시킨 연후에 대통령에 대한 특검 조사, 또 헌법 질서에 따른 조처를 하는 게 가장 중요하다"고 주장했다.이어 "새누리당이 모호한 입장을 견지하면서 정치권이 탄핵 절차에 착수하지 못하고 있다"면서 "새누리당 의원들께서 헌법기관으로서 자기 입장을 분명히 드러내야 한다"고 강조했다.김 의원에 따르면, 새누리당 유승민 의원은 지난 20일 비상시국회의에서 "하는데까지 해보자"면서 ▲박근혜 대통령 출당을 위한 윤리위 제소 ▲야당과 합의해 탄핵절차 착수 ▲ 야당이 요구할 경우 무조건 총리 임명 등 3가지를 제안했다.그는 "이 세 가지가 진행되면 사실 저와 남경필 지사가 탈당할 이유도 없었다"면서 "박 대통령의 윤리위 제소도 사실 하세월"이라고 지적했다.아울러 "탄핵 절차 착수를 먼저 해야 하기 때문에 3지대 보다는 야당과 새누리당 내에 있는 분들과 이야기를 집중해 나갈 생각"이라며 "이것이 진행된 이후에 새로운 보수의 중심을 어떻게 구축할 것인지에 대해 모든 가능성을 열어놓고 여러분들을 만나볼 생각"이라고 구상을 꺼냈다.김 의원은 지난 22일 남경필 경기도지사와 함께 새누리당을 탈당하면서 신당 창당 의지를 표명한 바 있다.그는 탈당 기자회견문에서 "진정한 보수의 중심을 세워 무엇보다도 먼저

In [7]:
print (titles)

['김용태 "비대위는 과정일 뿐 탄핵이 중요…입장 밝혀야"', '대통령 탄핵 절차 어떻게… A부터 Z까지', "박지원 vs 문재인, '박근혜 탄핵' 놓고 野 내분 격화", "'황교안 체제' 질겁하는 박지원, 송영길發 탄핵론에 동조", '문재인 "非朴 박근혜 탄핵해야", 김무성 "난 찬성표"', '권성동 "탄핵까지 4~6개월… \'헌재 증언\'할 사람 많아"', '권성동 "탄핵까지 4~6개월… \'헌재 증언\'할 사람 많아"', '권성동 "탄핵까지 4~6개월… \'헌재 증언\'할 사람 많아"', "'황교안 죽이기' 송영길 → 조배숙, 불편하면 제거?", "문재인, 대학생 '동맹휴업' 독려…국무위원엔 '항명' 선동", "문재인, 대학생 '동맹휴업' 독려…국무위원엔 '항명' 선동", '김무성 "대선 불출마"… 정계개편·개헌 염두', '非朴 탈당파, 국민의당과 접촉…"제4당의 태동 가능"', '문재인-박지원 설전 점입가경…"인신공격 도 넘었다"', '어제 장관들이 박원순 하자는대로 하였다면 쿠데타 성공!', '[충청브리핑] 충청에 ‘위암·대장암·폐암’ 왜 많은가?', '이정현, 南·金 나가자 입장 급선회… 왜?', '이정현 "나는 이제 썩은 거름…차기 비대위 논의해달라"', '[속보] 김용태·남경필 새누리당 탈당… 보수분열 현실화', '‘최순실 특검’ 유력 후보자 공통분모 ‘독수리5형제+우리법연구회’', '소용돌이치는 서청원~유승민 연대설… 실체는', '‘지지율 역전’ 박원순 시장, ‘이재명 따라하기’ 나섰지만', '[충청브리핑] 이승훈 청주시장 1심 당선 무효형 ‘술렁’', "민생·안보 뇌사…'배반의 국회' 국민이 심판할 때", "민생·안보 뇌사…'배반의 국회' 국민이 심판할 때", "'육룡이 나르샤' 월화드라마 선두 유지,'화려한 유혹'은 소폭하락", "'육룡이 나르샤' 월화드라마 선두 유지,'화려한 유혹'은 소폭하락", '박대통령이 야당의원들에게 그런 욕을 했다면...', '박대통령이 야당의원들에게 그런 욕을 했다면...', "신영복은 죽는 날까지 혁명 꿈을 버

In [8]:
print (reporters)

['임재섭', '정도원', '김현중', '김민우', '김현중', '임재섭', '임재섭', '임재섭', '김민우', '김현중', '김현중', '정도원', '김민우', '김현중', '편집자', '김정원', '임재섭', '임재섭', '정도원', '양원석', '정도원', '이길호', '김정원', '조우석', '조우석', '김상록', '김상록', '조화유', '조화유', '강철화', '강철화', '안종현', '한해선', '한해선', '전경웅', '조광형', '조광형', '조광형', '정도원', '김현중', '김용삼', '보성(', '조광형', '조광형', '이옥남', '김민우', '김현중', '김현중', '정도원', '이욱', '임혜진', '임혜진', '임혜진', '노민호', '노민호', '오창균', '강유화', '임재섭', '김민우', '김민우', '전경웅', '대담=', '김희진', '김현중', '김민우', '김민우', '김현중', '임재섭', '김현중', '김민우', '김민우', '김민우', '김민우', '김민우', '김민우', '김현중', '임재섭', '임재섭', '임재섭', '임재섭', '김현중', '이덕기', '이덕기', '이덕기', '김현중', '김현중', '김민우', '김민우', '정도원', '김성욱', '정도원', '정도원', '노민호', '노민호', '김현중', '정도원', '정도원', '인보길', '정도원', '임재섭', '임재섭', '임재섭', '임재섭', '임재섭', '임재섭', '김민우', '김민우', '임재섭', '김현중', '김현중', '김민우', '김민우', '김민우', '이길호', '이길호', '이길호', '인보길', '정도원', '안종현', '김현중', '정도원', '김민우', '전경웅', '임재섭', '임재섭', '정도원', '정도원', '정도원', '양원석', '김현중', '김민우', '허동혁', '김민우', '김민우', '임재섭', '김민우', '김현중', '박성현', '정도원', '이종현', '이종현', '김현중', '김현중',

In [24]:
rows = zip(id, sitenames, dates, times, reporters, titles, articles)

path = "/media/ubuntu4kee4/222E61A42E617229/Communication/탄핵/database/" + "뉴데일리.tsv"

with open(path, 'w') as f:
    for row in rows:
        f.write("".join(str(s)+"\t" for s in row)+ "\n")